
#  Building a minC (minimum C) compiler


Enter your name and student ID.

 * Name: Hiroto Uchida
 * Student ID: 03240406



<a name="intro"> </a>
# 1. Introduction
* This is the instruction for those who choose Option A for the term report
* You build a compiler for a minimum subset of C language, dubbed 'minC'
* This page explains how you should work on it, but <font color="red">your actual work is supposed to be done mostly in terminal and text editor</font>


# 2. Prepare AI Tutor
* execute the following cell to set up your tutor

In [1]:
from heytutor import *
config(default_lang="OCaml")  # choose one of Go/Julia/OCaml/Rust

* you may want to ask a few basics

In [3]:
hey("""ARM64で引数を読み込む時、アセンブリコードでx0とx1はどのように使用しますか、例えば x / 2/ 3を返す関数fはどのように記述されますか？""")

ARM64 のアセンブリコードで引数を読み込み、それを操作して結果を返す関数は、レジスタ `x0` や `x1` を通じて行います。以下は、関数 `f`（与えられた引数 `x` を `x/2/3` にして返す関数）がどのように記述されるかについての説明です。

---

### 想定される C 言語関数:
```c
int f(int x) {
    return x / 2 / 3;
}
```

この関数の動作は以下のステップを行います:
1. 入力は `x`（ここでは `int` 型）。
2. `x` を 2 で割り、その結果を更に 3 で割る。
3. 割り算の結果を返す。

`x` は ARM64 の calling convention に基づき、レジスタ `x0` に渡されます。戻り値も `x0` に格納されます。

---

### ARM64 アセンブリコード:

```assembly
// 引数: x (x0) を受け取り、結果を返す。
f:
    MOV x1, #2          // レジスタ x1 に 2 を格納
    SDIV x0, x0, x1     // x0 = x0 / x1 (x を 2 で割る)

    MOV x1, #3          // レジスタ x1 に 3 を格納
    SDIV x0, x0, x1     // x0 = x0 / x1 (結果を 3 で割る)

    RET                 // 関数を終了し、結果 (x0) を戻り値として返す
```

---

### 説明:

1. レジスタ `x0`:
   - 呼び出し規約により、引数 `x` が渡される。
   - 返す値も `x0` に格納される。

2. `MOV` 命令:
   - 定数（例えば 2 や 3）をワーク用レジスタ（`x1`）に格納する。
   - ARM64 アーキテクチャでは、割り算の除数をレジスタに保持する必要があるため、`MOV` で設定。

3. `SDIV` 命令（Signed Division）:
   - ARM64 の整数型除算命令。
   - 第一オペランドを第二オペランドで割り、その結果を先頭オペランドに格納する。
     - 例:`SDIV x0, x0, x1` は `x0 = x0 / x1` を意味する。

4. `RET` 命令:
   - 関数を終了し、戻り値として `x0` を呼び出し元へ返す。

---

### 呼び出し元コードの例:

この関数を呼び出すコードがコンパイラによって生成される場合、次のような手順で値を渡し、結果を受け取ります:

```assembly
MOV x0, #18        // 引数 x = 18 を設定
BL f               // 関数 f を呼び出し、戻り値が x0 に格納される
// x0 に結果 (18 / 2 / 3 = 3) が格納される
```

---

### 実行の流れ：

1. `x` が `x0` に渡される（例では `x = 18`）。
2. 関数内で `x/2` 計算（結果は 9）。
3. 次に `9/3` を計算（結果は 3）。
4. 最終結果（3）が `x0` に格納され、返される。

このアセンブリコードは、整数演算限定の例です。浮動小数点の場合には、`SDIV` の代わりに浮動小数点除算命令（`FDIV` 等）を使用します。

何か別の細かい点や質問があれば、ぜひ教えてくださいね！ 😊


# 3. Files
* `parser/` : a C -> XML parser written in Python + tatsu (parser generator), which converts a C source into an equivalent XML
* `{ml,jl,go,rs}/minc` : a directory for each language
* `test/` : test programs

# 4. Parser (C -> XML)
* `minc_grammar.y` : grammar definition of the minimum C language we work on, which is converted into a working python program (`minc_grammar.py`) using [tatsu](https://github.com/neogeny/TatSu) parser generator library

In [68]:
cd ~/notebooks/pl08_minc/parser
tatsu minc_grammar.y > minc_grammar.py

------------------------------------------------------------------------
         409  lines in grammar
          21  rules in grammar
         209  nodes in AST


* `minc_to_xml.py` : the main parser program that drives minc_grammar.py to convert a C source into an equivalent XML

In [69]:
python minc_to_xml.py example/ex.c > ex.xml

* examine the result and understand how each C construct is converted into XML

In [70]:
cat example/ex.c 

long f(long x, long y) {
  long u;
  u = x + y;
  return u * 3;
}


In [68]:
cat ex.xml

<program xmlns="https://program.com">
 <fun_def>
  <name>f</name>
  <params>
   <param>
    <type>
     <primitive_type>long</primitive_type>
    </type>
    <name>x</name>
   </param>
   <param>
    <type>
     <primitive_type>long</primitive_type>
    </type>
    <name>y</name>
   </param>
  </params>
  <return_type>
   <primitive_type>long</primitive_type>
  </return_type>
  <body>
   <compound>
    <decls>
     <decl>
      <type>
       <primitive_type>long</primitive_type>
      </type>
      <name>u</name>
     </decl>
    </decls>
    <stmts>
     <expr_stmt>
      <bin_op>
       <op>=</op>
       <left>
        <var>u</var>
       </left>
       <right>
        <bin_op>
         <op>+</op>
         <left>
          <var>x</var>
         </left>
         <right>
          <var>y</var>
         </right>
        </bin_op>
       </right>
      </bin_op>
     </expr_stmt>
     <return>
      <bin_op>
       <op>*</op>
       <left>
        <var>u</var>
       </left>
       <righ

* you don't have to modify `minc_grammar.y` or `minc_to_xml.py` unless you extend the grammar for extra points
* yet you are encouraged to see how simple does tatsu (or any parser generator, for that matter) make it to write a parser; just take a look at `minc_grammar.y`

##  Note: why are we using XML?
* it is unnecessary and unusual to convert the source program first into XML and then to the abstract syntax tree
* more commonly, you use a parser generator for the language you write your compiler with (OCaml, Julia, Go, or Rust), which allows you to directly build the abstract syntax tree you can manipulate in that language
* for example, C/C++ has [flex](https://en.wikipedia.org/wiki/Flex)/[bison](https://en.wikipedia.org/wiki/GNU_Bison) parser generator, whose grammar description file (analogous to minc_grammar.y) allows you to build any C/C++ data structure in it; OCaml has [ocamllex/ocamlyacc](https://v2.ocaml.org/manual/lexyacc.html) ([Menhir](http://gallium.inria.fr/~fpottier/menhir/) is a newer version of ocamlyacc).  there is a parser generator that supports multiple languages, most notably [ANTLR](https://www.antlr.org/), which supports Java and Python code generation.  in circumstances where there is a tool available for your language, it is much more straightforward and convenient to use these tools without going through XML
* the reasons why we go through XML are
  * I could not find a popular parser generator for some of the languages (Go and Julia)
  * even if one exists in each language, there will be differences between them that make it difficult/tricky/cumbersome to explain them
* so I arrived at a parser generator (tatsu) for Python as a common middle ground and XML as a common data structure all languages can easily read


# 5. {go,jl,ml,rs}/minc
* in each language-specific directory (`go, jl, ml, rs`) , there is a toplevel directory `minc`
* the code given there is a skeleton of a compiler that reads an XML file, builds its abstract syntax tree, and finally calls the code generator
* the code generator is currently almost empty and raises an exception when called
* your main job in the assignment is to complete the code generator

## 5-1. Go
###  files
* `go/`
  * `minc/`
    * `minc_ast.go` --- abstract syntax tree (AST) definition
    * `minc_parse.go` --- XML -> AST converter
    * `minc_cogen.go` --- AST -> assembly code
    * `minc.go` --- the main file

###  build

In [ ]:
export PATH=~/go/bin:$PATH
cd ~/notebooks/pl08_minc/go/minc
ls -lR

In [ ]:
go build

###  run
* be sure you have generated ex.xml by `minc_to_xml.py`
* try to compile a small program and see that the code generator raises an exception


In [ ]:
./minc ../../parser/ex.xml ex.s

* take a look at the source code that caused the exception

In [ ]:
cat minc_cogen.go

* your job is to implement `ast_to_asm_program` function

## 5-2. Julia
###  files
* `jl/`
  * `minc/`
    * `minc_ast.jl` --- abstract syntax tree (AST) definition
    * `minc_parse.jl` --- XML -> AST converter
    * `minc_cogen.jl` --- AST -> assembly code
    * `minc.jl` --- the main file

###  build

In [ ]:
export PATH=~/.juliaup/bin:$PATH
cd ~/notebooks/pl08_minc/jl/minc
ls -lR

In [ ]:
chmod +x minc.jl

###  run
* be sure you have generated ex.xml by `minc_to_xml.py`
* try to compile a small program and see that the code generator raises an exception


In [ ]:
./minc.jl ../../parser/ex.xml ex.s

* take a look at the source code that caused the exception

In [ ]:
cat minc_cogen.jl

* your job is to implement `ast_to_asm_program` function

## 5-3. OCaml
###  files
* `ml/`
  * `minc/`
    * `libs/`
      * `minc_ast.ml` --- abstract syntax tree (AST) definition
      * `minc_parse.ml` --- XML -> AST converter
      * `minc_cogen.ml` --- AST -> assembly code
      * `dune` --- describes dependencies between them
    * `bin/`
      * `main.ml` --- the main file

###  build

In [13]:
cd ../../test
pwd
python3 ../parser/minc_to_xml.py src/f31.c > xml/f31.xml && cat xml/f31.xml

bash: cd: ../../test: No such file or directory
/home/u25067/notebooks/pl08_minc/test
<program xmlns="https://program.com">
 <fun_def>
  <name>f</name>
  <params>
   <param>
    <type>
     <primitive_type>long</primitive_type>
    </type>
    <name>x</name>
   </param>
  </params>
  <return_type>
   <primitive_type>long</primitive_type>
  </return_type>
  <body>
   <compound>
    <decls/>
    <stmts>
     <return>
      <bin_op>
       <op>/</op>
       <left>
        <bin_op>
         <op>/</op>
         <left>
          <var>x</var>
         </left>
         <right>
          <int_literal>2</int_literal>
         </right>
        </bin_op>
       </left>
       <right>
        <int_literal>3</int_literal>
       </right>
      </bin_op>
     </return>
    </stmts>
   </compound>
  </body>
 </fun_def>
</program>



In [285]:
eval $(opam env)
cd ~/notebooks/pl08_minc/ml/minc
# ls -lR

In [286]:
# dune init project minc
dune build

Command 'dune' not found, but can be installed with:
apt install ocaml-dune
Please ask your administrator.


: 127

###  run
* be sure you have generated ex.xml by `minc_to_xml.py`
* try to compile a small program and see that the code generator raises an exception


In [3]:
_build/default/bin/main.exe ../../parser/ex.xml ex.s

* take a look at the source code that caused the exception

In [57]:
cat lib/minc_cogen.ml

* your job is to implement `ast_to_asm_program` function

## 5-4. Rust
###  files
* `rs/`
  * `minc/`
    * `src/`
      * `minc_ast.rs` --- abstract syntax tree (AST) definition
      * `minc_parse.rs` --- XML -> AST converter
      * `minc_cogen.rs` --- AST -> assembly code
      * `main.rs` --- the main file

###  build

In [180]:
. ~/.cargo/env
cd ~/notebooks/pl08_minc/rs/minc
ls -lR

.:
total 12
-rw-r--r-- 1 u25067 u25067 2353 Jun 23 00:55 Cargo.lock
-rw-r--r-- 1 u25067 u25067  187 Jun 23 00:55 Cargo.toml
drwxr-xr-x 2 u25067 u25067 4096 Jun 23 00:55 src

./src:
total 32
-rw-r--r-- 1 u25067 u25067   963 Jun 23 00:55 main.rs
-rw-r--r-- 1 u25067 u25067  5561 Jun 23 00:55 minc_ast.rs
-rw-r--r-- 1 u25067 u25067   375 Jun 23 00:55 minc_cogen.rs
-rw-r--r-- 1 u25067 u25067 13184 Jun 23 00:55 minc_parse.rs


In [ ]:
cargo build

###  run
* be sure you have generated ex.xml by `minc_to_xml.py`
* try to compile a small program and see that the code generator raises an exception


In [32]:
./target/debug/minc ../../parser/ex.xml ex.s

bash: ./target/debug/minc: No such file or directory


: 127

* take a look at the source code that caused the exception

In [ ]:
cat src/minc_cogen.rs

* your job is to implement `ast_to_asm_program` function


# 6. test
* your primary goal is to pass all the tests

##  files
* `test/`
  * `src/` 
    * `f00.c, f01.c, f02.c, ..., ` --- test programs each definint a function `f`
  * `main.c` --- a file that calls function `f`
  * `Makefile` --- executes all the tests

the `Makefile` does the following for each test program (`src/f00.c, src/f01.c,` ...)

1. convert `src/fXX.c` to `xml/fXX.xml` with `parser/minc_to_xml.py`
1. compile `xml/fXX.xml` to `minc/fXX.s` with the minC compiler you are supposed to write
1. compile `main.c` and `minc/fXX.s` into an executable
1. compile `main.c` and `src/fXX.c` into an executable with gcc
1. execute the two executables and compare their outputs

* in the `Makefile`
 * <font color=red>you must set the variable `minc` to the path to your compiler (relative to the test directory)</font>
```
minc := ../ml/minc/_build/default/bin/main.exe
```
 * you can set the variable `srcs` to change the functions tested. the default is all the 75 files in `src/`
```
srcs := $(wildcard src/f*.c)
```
for example, if you set this to
```
srcs := $(wildcard src/f00.c)
```
you can only test `src/f00.c`

 * `make -n` will show you what will be executed without actually executing it
 

In [100]:
# make sure you set minc variable in Makefile
#cd ~/notebooks/pl08_minc/test
#make -n

to test a single file

In [101]:
#make -n srcs=src/f00.c


* if the test fails, identify which file it failed for and how

* use
```
make srcs=src/fXX.c
```
to run the test on the particular file that failed

* on each file (`src/fXX.c`), it first converts the C source file into XML using `minc_to_xml.py`; if the test fails here, it's not your fault, unless you modified the grammar

```
echo "# convert src/f00.c to xml/f00.xml"
../parser/minc_to_xml.py src/f00.c > xml/f00.xml
```

* next it calls your compiler to convert the XML file into asm (the command line depends on the language you chose)

```
echo "# compile xml/f00.xml to asm with your minC compiler"
../ml/minc/_build/default/bin/main.exe xml/f00.xml asm/f00.s
```

if this fails, examine the original C source file (`f00.c` in the case above) and XML source file to examine what kind of source code makes it fail

* then it calls the gcc to compile the assembly you generated into an executable

```
echo "# generate the executable that calls f with your minC compiler"
gcc -o minc/f00.exe -DTEST_NO=0 main.c asm/f00.s -O0 -g
```

if you generate a syntactically invalid assembly code, gcc will complain here.  read the gcc error message, examine the assembly you generated (`asm/f00.s` in the case above) and understand why it failed

* then it calls executables generated by gcc as well as your minC compiler

```
echo "# run the executable generated by gcc"
./gcc/f00.exe | tee out/f00.gcc
echo "# run the executable generated by your minC compiler"
./minc/f00.exe | tee out/f00.minc
```
the gcc-generated executable is unlikely to fail.  if your compiler-generated executable fails, you might be able to see the reason just by looking at the assembly code you generated; otherwise, run the executable with a debugger. GDB, for example, can step-execute an assembly program and allow you to examine the value of registers.

* it finally compares the output of the two executables

```
echo "# take the diff of the two"
diff out/f00.gcc out/f00.minc
```

if it fails, the debugging strategy is the same as above; you might be able to see the reason just by looking at the assembly code you generated; otherwise, run the executable with a debugger. GDB, for example, can step-execute an assembly program and allow you to examine the value of registers.

## 6-1. How to add new test programs
* if you work on any of the extension, you are likely to add test programs too
* src/src/fun.c contains all the C functions to test, each one being guarded by
```
#if TEST_NO == ???

#endif
```
add your test case at the end of the file, using the next number as your TEST_NO
* modify the following line in `test/src/src/Makefile`
```
tests := $(shell seq -f "%02.0f" 0 74)
```
to reflect the tests you added.  for example, if you add two test cases (TEST_NO = 75 and 76), you should change it to 
```
tests := $(shell seq -f "%02.0f" 0 76)
```
and run make in `test/src/src`

In [67]:
cd ~/notebooks/pl08_minc/test/src/src
make

make: Nothing to be done for 'all'.


* do a similar thing for `test/main.c`; as long as all functions take only `long` arguments and return a `long` value, you can use the same `main` function. in that case you just change
```
#if 0 <= TEST_NO && TEST_NO <= 74
...
#endif
```
to, say, 
```
#if 0 <= TEST_NO && TEST_NO <= 76
...
#endif
```

* if you support types other than `long`, you are likely to add a different main function. modify `main.c` accordingly



# 7. Format of the report and how to submit your work
## 7-1. Baseline requirements (Level 1)
* implement the compiler for minC (you will mainly write `minc_cogen.{go,jl,ml,rs}`)
* your code generator must be heavily commented (explain how it works, as if you are writing a report, except you write it in the source code)
* pass all the tests, that is, the following command executes without an error

In [1]:
eval $(opam env)

In [41]:
eval $(opam env)
ocamlc --version
cd ~/notebooks/pl08_minc/ml/minc
# ls -lR

4.14.2


In [42]:
# dune init project minc
dune build

In [43]:
cd ~/notebooks/pl08_minc/test
make -B

mkdir -p xml/dir
# convert src/f00.c to xml/f00.xml
python3 ../parser/minc_to_xml.py src/f00.c > xml/f00.xml
mkdir -p asm/dir
# compile xml/f00.xml to asm with your minC compiler
../ml/minc/_build/default/bin/main.exe xml/f00.xml asm/f00.s
mkdir -p minc/dir
# generate the executable that calls f with your minC compiler
gcc -o minc/f00.exe -DTEST_NO=0 main.c asm/f00.s -O0 -g
mkdir -p out/dir
# run the executable generated by gcc
minc/f00.exe | tee out/f00.minc
123
mkdir -p gcc/dir
# generate the executable that calls f with gcc
gcc -o gcc/f00.exe -DTEST_NO=0 main.c src/f00.c -O0 -g
# run the executable generated by gcc
gcc/f00.exe | tee out/f00.gcc
123
# take the diff of the two
diff out/f00.gcc out/f00.minc > out/f00.diff
# convert src/f01.c to xml/f01.xml
python3 ../parser/minc_to_xml.py src/f01.c > xml/f01.xml
# compile xml/f01.xml to asm with your minC compiler
../ml/minc/_build/default/bin/main.exe xml/f01.xml asm/f01.s
# generate the executable that calls f with your minC compiler

: 2

In [147]:
gcc -o minc/f17.exe -DTEST_NO=17 main.c asm/f17.s -O0 -g

In [16]:
make -n srcs=src/f10.c

echo "# compile xml/f10.xml to asm with your minC compiler"
../ml/minc/_build/default/bin/main.exe xml/f10.xml asm/f10.s
echo "# generate the executable that calls f with your minC compiler"
gcc -o minc/f10.exe -DTEST_NO=10 main.c asm/f10.s -O0 -g
echo "# run the executable generated by gcc"
minc/f10.exe | tee out/f10.minc
echo "# take the diff of the two"
diff out/f10.gcc out/f10.minc > out/f10.diff


* by default, the entire test stops as soon as any test fails. you may want to try `-k` option for make, to skip files that fails and go ahead to others

In [6]:
cd ~/notebooks/pl08_minc/test
make -B -k

mkdir -p xml/dir
# convert src/f00.c to xml/f00.xml
python3 ../parser/minc_to_xml.py src/f00.c > xml/f00.xml
mkdir -p asm/dir
# compile xml/f00.xml to asm with your minC compiler
../ml/minc/_build/default/bin/main.exe xml/f00.xml asm/f00.s
mkdir -p minc/dir
# generate the executable that calls f with your minC compiler
gcc -o minc/f00.exe -DTEST_NO=0 main.c asm/f00.s -O0 -g
mkdir -p out/dir
# run the executable generated by gcc
minc/f00.exe | tee out/f00.minc
123
mkdir -p gcc/dir
# generate the executable that calls f with gcc
gcc -o gcc/f00.exe -DTEST_NO=0 main.c src/f00.c -O0 -g
# run the executable generated by gcc
gcc/f00.exe | tee out/f00.gcc
123
# take the diff of the two
diff out/f00.gcc out/f00.minc > out/f00.diff
# convert src/f01.c to xml/f01.xml
python3 ../parser/minc_to_xml.py src/f01.c > xml/f01.xml
# compile xml/f01.xml to asm with your minC compiler
../ml/minc/_build/default/bin/main.exe xml/f01.xml asm/f01.s
# generate the executable that calls f with your minC compiler

* write the summary of tests that passed/failed, in the online Excel `teamXX/pl08_minc/results.xlsx`
 * P : passed
 * C : cannot assemble (your compiler fails to produce assembly)
 * I : invalid assembly (your compiler produced an assembly code but it does not compile with gcc)
 * R : runtime error (your compiler produced an executable, but it does not terminate successfully)
 * W : wrong result (your compiler produced an executable that terminates successfully, but the output result is different from that of gcc-generated executable)

* members of each team must get togeter at least once to show their test results each other
* you are encouraged to get together with team members working on this option in other occassions to discuss progress and help each other (team members working on other options are not requierd to participate)


* <font color=red>before submitting your work, make sure you clean up your working directory,</font>  in the manner described in [Writing standalone programs using libraries](https://taura.github.io/programming-languages/html/errata/pl04_standalone.sos.html)

* Go

In [ ]:
cd ~/notebooks/pl08_minc/go/minc
go clean

* OCaml
  * ignore `Error: rmdir(_build): Directory not empty` if you see it (I don't know why it happens)
  

In [ ]:
cd ~/notebooks/pl08_minc/ml/minc
dune clean

* Rust

In [97]:
cd ~/notebooks/pl08_minc/rs/minc
cargo clean

Command 'cargo' not found, but can be installed with:
snap install rustup  # version 1.27.1, or
apt  install cargo   # version 1.75.0+dfsg0ubuntu1-0ubuntu7.1
apt  install rustup  # version 1.26.0-3
See 'snap info rustup' for additional versions.


: 127

* <font color="red">also make sure you execute `make clean` in the test directory</font>

In [ ]:
cd ~/notebooks/pl08_minc/test
make clean

* submit your work through Jupyter (modify and add files in place under `~/notebooks/pl08_minc`); make sure you execute the above cell (`make -B -k`)
* submit "Term Report Option A (pl08_minc; build a compiler)" through UTOL
* all the essential work is submitted through Juptyer and Excel; you only have to submit a brief report to UTOL

## 7-2. Extra points [Level 2+]
* you get extra points by doing more than required above

* you can extend the minC language in any ways, but possible extensions include (but are not limited to):
  * syntactic extensions
    * [difficulty level 2] for loop
    * [difficulty level 2.5] initializing declaration (e.g., int x = y + 2)
  * type extensions and type checks
    * note that supporting a type other than long requires the compiler know the type of an expression; it's a heavy lifting
    * [difficulty level 3] pointers (long*, long**, etc.)
    * [difficulty level 4] floating point numbers (double)
    * [difficulty level 5] types of different sizes (int, float)
    * [difficulty level 6] structures and typedefs
  * optimization
    * use registers more aggressively
    * inline-expand function calls

* if you have done extra work beyond requirements, describe what you did in the extra.docx. clearly indicate the author of each section (who did what)